# Finetune LLM with LLaMA Factory

> Add blockquote



Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

## Step 1: Install Dependencies

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 475, done.
remote: Counting objects: 100% (475/475), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 475 (delta 122), reused 294 (delta 73), pack-reused 0 (from 0)
Receiving objects: 100% (475/475), 5.17 MiB | 17.65 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/LLaMA-Factory
assets/       docker/    Makefile        README.md         scripts/  tests/
CITATION.cff  examples/  MANIFEST.in     README_zh.md      setup.py  tests_v1/
data/         LICENSE    pyproject.toml  requirements.txt  src/
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.4.dev0-0.editable-py3-non

## Step 2: Register Dataset

In [ ]:
import json
import os

# 1. Use the ABSOLUTE path so it never gets lost
# (This assumes you are on Google Colab defaults)
config_path = "/content/LLaMA-Factory/data/dataset_info.json"
dataset_path = "/content/LLaMA-Factory/data/asset_management_broad_v2.json"

# Debugging: Verify the files actually exist before trying to open
if not os.path.exists(config_path):
    print(f"❌ Error: Config file not found at {config_path}")
    print("   Please verify you cloned the repo successfully.")
else:
    print(f"✅ Found config file at: {config_path}")

if not os.path.exists(dataset_path):
    print(f"❌ Error: Your JSON file is missing at {dataset_path}")
    print("   Make sure you dragged 'asset_management_broad_v2.json' into the 'data' folder!")
else:
    print(f"✅ Found your dataset at: {dataset_path}")

# 2. Only proceed if files exist
if os.path.exists(config_path) and os.path.exists(dataset_path):
    with open(config_path, "r", encoding="utf-8") as f:
        data_info = json.load(f)

    # 3. Add your specific dataset to the list
    data_info["asset_management_broad_v2"] = {
        "file_name": "asset_management_broad_v2.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output"
        }
    }

    # 4. Save the file back
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(data_info, f, indent=2)

    print("\n🎉 Success! 'asset_management_train' is registered.")

✅ Found config file at: /content/LLaMA-Factory/data/dataset_info.json
✅ Found your dataset at: /content/LLaMA-Factory/data/asset_management_broad_v2.json

🎉 Success! 'asset_management_train' is registered.


## Step 3: Connect Hugging Face Token to Notebook

In [ ]:
from huggingface_hub import login

# Paste your token inside the quotes
login(token="insert token")

## Step 4: Fine-tune model via LLaMA Board. Run cell and select "Running on public URL".

In [ ]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2025-12-09 09:56:01.556765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765274161.576515    1673 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765274161.582448    1673 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765274161.597767    1673 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765274161.597797    1673 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765274161.597801    1673 computation_placer.cc:177]

# Extra Features from Original Notebook

### Check GPU environment

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Update Identity Dataset

In [ ]:
import json

%cd /content/LLaMA-Factory/

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

## Fine-tune model via Command Line

It takes ~30min for training.

In [ ]:
import json

args = dict(
  stage="sft",                                               # do supervised fine-tuning
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="identity,alpaca_en_demo",                         # use alpaca and identity datasets
  template="llama3",                                         # use llama3 prompt template
  finetuning_type="lora",                                    # use LoRA adapters to save memory
  lora_target="all",                                         # attach LoRA adapters to all linear layers
  output_dir="llama3_lora",                                  # the path to save LoRA adapters
  per_device_train_batch_size=2,                             # the micro batch size
  gradient_accumulation_steps=4,                             # the gradient accumulation steps
  lr_scheduler_type="cosine",                                # use cosine learning rate scheduler
  logging_steps=5,                                           # log every 5 steps
  warmup_ratio=0.1,                                          # use warmup scheduler
  save_steps=1000,                                           # save checkpoint every 1000 steps
  learning_rate=5e-5,                                        # the learning rate
  num_train_epochs=3.0,                                      # the epochs of training
  max_samples=500,                                           # use 500 examples in each dataset
  max_grad_norm=1.0,                                         # clip gradient norm to 1.0
  loraplus_lr_ratio=16.0,                                    # use LoRA+ algorithm with lambda=16.0
  fp16=True,                                                 # use float16 mixed precision training
  report_to="none",                                          # disable wandb logging
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

## Infer the fine-tuned model

In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",                        # load the saved LoRA adapters
  template="llama3",                                         # same to the one in training
  finetuning_type="lora",                                    # same to the one in training
)
chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

## Merge the LoRA adapter and optionally upload model

NOTE: the Colab free version has merely 12GB RAM, where merging LoRA of a 8B model needs at least 18GB RAM, thus you **cannot** perform it in the free version.

In [ ]:
!huggingface-cli login

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",                       # load the saved LoRA adapters
  template="llama3",                                        # same to the one in training
  finetuning_type="lora",                                   # same to the one in training
  export_dir="llama3_lora_merged",                          # the path to save the merged model
  export_size=2,                                            # the file shard size (in GB) of the merged model
  export_device="cpu",                                      # the device used in export, can be chosen from `cpu` and `auto`
  # export_hub_model_id="your_id/your_model",               # the Hugging Face hub ID to upload model
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json